<a href="https://colab.research.google.com/github/MaksPlank/Dummy/blob/master/PandasCase_Quizfreeze_unit_38_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

from google.colab import drive
# drive.mount('/content/gdrive')

events_df = pd.read_csv('/content/gdrive/MyDrive/DARP-1014/PROJECT_3_Python/events.csv', sep=',')
purchase_df = pd.read_csv('/content/gdrive/MyDrive/DARP-1014/PROJECT_3_Python/purchase.csv', sep=',')

### ФОРМИРОВАНИЕ ВЫБОРКИ [events_set2018]
из таблицы events_df выбираем все события для пользователей зарегестрированныx в 2018г

In [6]:
# ФОРМИРОВАНИЕ ВЫБОРКИ [events_set2018]

# 'start_time' в формат datetime:
time = pd.to_datetime(events_df['start_time'], dayfirst=True, errors ='coerce')
events_df['start_time'] = time

# usersid_reg2018 - список пользователей зарегестрированыx в 2018г
users_reg2018 = events_df[(events_df['event_type']=='registration')
                          &(events_df['start_time']>='2018-01-01')
                          &(events_df['start_time']<'2019-01-01')]['user_id']

# users_2018 - DataFrame (19926 rows x 2 columns)
users_2018 = pd.DataFrame({'user_id': users_reg2018,'year_reg': 2018})

# events_set2018 - искомая выборка (66959 rows x 7 columns)
events_set2018 = events_df.merge(users_2018, how='inner', on='user_id')

# events_set2018[events_set2018['event_type'] == 'level_choice'].info()
# events_set2018[events_set2018['event_type'].isin(['tutorial_start','tutorial_finish'])].info()
# events_set2018.describe(include='all')
events_set2018['event_type'].unique()

rows_count = events_set2018['id'].count()
user_count_uniq = events_set2018['user_id'].nunique()
selected_level_count = events_set2018[events_set2018['event_type'] == 'level_choice']['selected_level'].count()

print('\n   РЕЗЮМИРУЕМ: ')
print(f'▪ {rows_count} строк в датасете events_set2018')
print(f'▪ {user_count_uniq} уникальных пользователей совершившали действия')
print(f'▪ {selected_level_count} непустых значения в столбце selected_level')
print(f'▪ признак выбора уровня сложности (selected_level) может принимать значения: easy, medium, hard. Или отсутствовать, nan')


   РЕЗЮМИРУЕМ: 
▪ 66959 строк в датасете events_set2018
▪ 19926 уникальных пользователей совершившали действия
▪ 8342 непустых значения в столбце selected_level
▪ признак выбора уровня сложности (selected_level) может принимать значения: easy, medium, hard. Или отсутствовать, nan


### ФОРМИРОВАНИЕ ВЫБОРКИ [purchase_set2018]
из таблицы purchase_df выбираем все события для пользователей зарегестрированныx в 2018г

In [7]:
# ФОРМИРОВАНИЕ ВЫБОРКИ [purchase_set2018]

# 'start_time' в формат datetime:
date = pd.to_datetime(purchase_df['event_datetime'], dayfirst=True)
purchase_df['event_datetime'] = date

# purchase_df_set2018 - искомая выборка (1600 rows x 5 columns)
purchase_set2018 = purchase_df.merge(users_2018, how='inner', on='user_id')
purchase = purchase_set2018.copy()

# purchase_set2018.describe(include='all')
lines = purchase_set2018['user_id'].nunique()
amount_mean = purchase_set2018['amount'].mean().round(2)

print('\n   РЕЗЮМИРУЕМ: ')
print(f'▪ {lines} cтрок в датафрейме purchase_df')
print(f'▪ {lines} уникальных пользователей в датафрейме')
print(f'▪ {amount_mean} - среднее значение в столбце amount')
print(f'▪ в датафрейме purchase_set2018 нет пропусков')


   РЕЗЮМИРУЕМ: 
▪ 1600 cтрок в датафрейме purchase_df
▪ 1600 уникальных пользователей в датафрейме
▪ 110.73 - среднее значение в столбце amount
▪ в датафрейме purchase_set2018 нет пропусков


### 1. АНАЛИЗ ПОЛЬЗОВАТЕЛЬСКИХ СОБЫТИЙ
определим как пользователи переходят из этапа в этап, определим конверсию на каждом из этапов.
т.е. пределим какой процент пользователей переходит с предыдущего этапа на следующий

In [8]:
# АНАЛИЗ ПОЛЬЗОВАТЕЛЬСКИХ СОБЫТИЙ

# расчет числа пользователей, совершивших целевые действия :
reg = events_set2018[events_set2018['event_type'] == 'registration']['user_id'].nunique()       # 19926
start = events_set2018[events_set2018['event_type'] == 'tutorial_start']['user_id'].nunique()   # 11858
lvl = events_set2018[events_set2018['event_type'] == 'level_choice']['user_id'].nunique()       # 8342
pack = events_set2018[events_set2018['event_type'] == 'pack_choice']['user_id'].nunique()       # 5737
finish = events_set2018[events_set2018['event_type'] == 'tutorial_finish']['user_id'].nunique() # 10250

# users_uniq - 100% пользователей - 19926
users_uniq = events_set2018['user_id'].nunique()
# users_uniq - 100% покупателей  - 1600
purchase_uniq = purchase_set2018['user_id'].nunique()

# расчет доли,% действий
reg_percent = reg/users_uniq      # 100%
start_percent = start/users_uniq   # 59.51%
lvl_percent = lvl/reg             # 41.86%
pack_percent = pack/lvl           # 68.77%
finish_percent = finish/start     # 86.44%
purch_pack_percent = purchase_uniq/pack   # 27.89%
purch_reg_percent = purchase_uniq/reg     # 8.03%

print('\n   ЭТАПЫ, количество чел.')
print(f'{reg} - зарегестрировались(уникальные пользователи)')
print(f'{start} - начали обучение')
print(f'{lvl}  - выбрали уровень сложности')
print(f'{pack}  - выбрали бесплатные пакеты')
print(f'{finish} - закончили обучение')
print(f'{purchase_uniq}  - купили платную версию')

print('\n   ЭТАПЫ, конверсия,% ')
print(f'регестрация ➝ {start_percent:.2%} ➝ начало обучения ➝ {finish_percent:.2%} ➝ завершение обучения')
print(f'регестрация ➝ {lvl_percent:.2%} ➝ выбор уровня сложности ➝ {pack_percent:.2%} ➝ выбор бесплатных вопросов')
print(f'регестрация ➝ {purch_reg_percent:.2%} ➝ покупка ')
print(f'выбор бесплатных вопросов ➝ {purch_pack_percent:.2%} ➝ покупка ')

print('\n   РЕЗЮМИРУЕМ: ')
print(f'▪ {reg_percent:.0%} пользователей({reg} чел.) регистрируются - это обязательное действие')
print(f'▪ {start} пользователей начинают обучение, это {start_percent:.2%} от зарегестрировавшихся')
print(f'▪ {lvl} пользователей выбрали уровень сложности, это {lvl_percent:.2%} от числа зарегестрировавшихся,')
print(f'        т.е. меньше половины пользователей подходят на шаг ближе к монетизации.')
print(f'▪ {pack} пользователей выбрали пакеты бесплатных вопросов, это {pack_percent:.2%} от числа пользователей, выбравших уровень сложности.')
print(f'▪ {finish} пользователей завершили обучение, это {finish_percent:.2%} от числа начавших. Это достаточно хороший процент прохождения обучения')
print(f'▪ {purchase_uniq} пользователей сделали покупку, это {purch_pack_percent:.2%} от числа пользователей, которые выбрали бесплатные вопросы')
print(f'  {purch_reg_percent:.2%} от числа зарегистрировавшихся пользователей')



   ЭТАПЫ, количество чел.
19926 - зарегестрировались(уникальные пользователи)
11858 - начали обучение
8342  - выбрали уровень сложности
5737  - выбрали бесплатные пакеты
10250 - закончили обучение
1600  - купили платную версию

   ЭТАПЫ, конверсия,% 
регестрация ➝ 59.51% ➝ начало обучения ➝ 86.44% ➝ завершение обучения
регестрация ➝ 41.86% ➝ выбор уровня сложности ➝ 68.77% ➝ выбор бесплатных вопросов
регестрация ➝ 8.03% ➝ покупка 
выбор бесплатных вопросов ➝ 27.89% ➝ покупка 

   РЕЗЮМИРУЕМ: 
▪ 100% пользователей(19926 чел.) регистрируются - это обязательное действие
▪ 11858 пользователей начинают обучение, это 59.51% от зарегестрировавшихся
▪ 8342 пользователей выбрали уровень сложности, это 41.86% от числа зарегестрировавшихся,
        т.е. меньше половины пользователей подходят на шаг ближе к монетизации.
▪ 5737 пользователей выбрали пакеты бесплатных вопросов, это 68.77% от числа пользователей, выбравших уровень сложности.
▪ 10250 пользователей завершили обучение, это 86.44% от чи

### 2. АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ
определим, какие есть различные последовательности прохождения пользователями этапов обучения
насколько часто они встречаются, и какая часть их приходит к оплате

In [9]:
# АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

# 1. Объединение таблиц событий и оплат
#    Создание нового маркера для признака покупки
#    Переименование разные по смыслу признаки id в таблицах
#    Соединение таблиц методом конкатанации
purchase_set2018['event_type'] = 'purchase'
events_set2018 = events_set2018.rename(columns={'id': 'event_id'})
purchase_set2018 = purchase_set2018.rename(columns={'id': 'purchase_id', 'event_datetime': 'start_time'})
total_events_df = pd.concat([events_set2018,purchase_set2018],sort=False)


# 2. Сброс индексов. Сортировкка всех событий по возрастанию даты/времени.
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
print('\ntotal_events_df \nCводная таблица событий и оплат для пользователей зарегестрированных в 2018г:')
display(total_events_df)


# 3. Объединение всеx событий в один список для каждого пользователя
user_path_df = (total_events_df.groupby(['user_id'])['event_type'].apply(list).reset_index())
user_path_df['event_path'] = user_path_df['event_type'].apply(lambda x: " > ".join(x))
user_path_df = user_path_df.drop(['event_type'],axis=1)
print('\nuser_path_df \nТаблица всеx событий для каждого пользователя :')
display(user_path_df)


# 4. Подсчет частоты различных списков последовалельностей
#    Группировка по event_path
#    Сортировка от самого популярного к наименее популярному
#    Вывод количества повторов
#    Вывод 10 наиболее популярных последовательности, содержащих оплату
user_paths = (user_path_df.groupby(['event_path'])['user_id'].nunique().sort_values(ascending=False))
perch10 = user_paths[user_paths.index.str.contains('purchase')].head(10)

# 5. Дополнительные переменные
pop_perch = user_paths[4]                      # 1083 чел.
perch_percent_reg = pop_perch/reg              # 5.44%
perch_percent_start = pop_perch/start          # 9.13%
perch_percent_perch = pop_perch/purchase_uniq  # 67.69%



print('\n РЕЗЮМИРУЕМ: ')
print(f'▪ Среди 10 самых популярных последовательностей только одна содержит этап оплаты. Это последовательность:')
print('   registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase')
print(f'  по этому пути прошли {pop_perch} пользователей, это {perch_percent_perch:.2%} всех оплат')
print(f'  но всего лишь {perch_percent_start:.2%} начавших обуение\n')
print(f'▪ Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения.')
print('  Отсюда следует гипотеза: вероятность оплаты зависит от того, проходил ли пользователь бесплатное обучение.')


total_events_df 
Cводная таблица событий и оплат для пользователей зарегестрированных в 2018г:


,event_id,event_type,selected_level,start_time,tutorial_id,user_id,year_reg,purchase_id,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,2018,NaN,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,2018,NaN,NaN
4,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,2018,NaN,NaN
7,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,2018,NaN,NaN
12,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,2018,NaN,NaN
...,...,...,...,...,...,...,...,...,...
68554,NaN,purchase,NaN,2019-01-02 03:48:19,NaN,47498,2018,18441.0,100.0
68555,NaN,purchase,NaN,2019-01-02 23:26:26,NaN,47647,2018,18442.0,150.0
68556,NaN,purchase,NaN,2019-01-03 00:36:36,NaN,47554,2018,18443.0,50.0
68557,NaN,purchase,NaN,2019-01-04 12:51:41,NaN,47742,2018,18448.0,50.0



user_path_df 
Таблица всеx событий для каждого пользователя :


,user_id,event_path
0,27832,registration
1,27833,registration > tutorial_start > tutorial_finish
2,27834,registration > tutorial_start > tutorial_finish
3,27835,registration > tutorial_start > tutorial_finis...
4,27836,registration > tutorial_start > tutorial_start...
...,...,...
19921,47753,registration > tutorial_start > tutorial_finis...
19922,47754,registration
19923,47755,registration > tutorial_start > tutorial_finis...
19924,47756,registration > tutorial_start > tutorial_finis...



 РЕЗЮМИРУЕМ: 
▪ Среди 10 самых популярных последовательностей только одна содержит этап оплаты. Это последовательность:
   registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase
  по этому пути прошли 1083 пользователей, это 67.69% всех оплат
  но всего лишь 9.13% начавших обуение

▪ Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения.
  Отсюда следует гипотеза: вероятность оплаты зависит от того, проходил ли пользователь бесплатное обучение.


### 3. АНАЛИЗ ВРЕМЕННЫХ ПРОМЕЖУТКОВ
определим среднее время, которое нужно пользователям для совершения целевого действия (оплаты)

In [10]:
# АНАЛИЗ ВРЕМЕННЫХ ПРОМЕЖУТКОВ

# 1. Выборка всех событий типа
reg_df = total_events_df[total_events_df['event_type'] == 'registration']
level_ch_df = total_events_df[total_events_df['event_type'] == 'level_choice']
pack_ch_df = total_events_df[total_events_df['event_type'] == 'pack_choice']
purch_df = total_events_df[total_events_df['event_type'] == 'purchase']
start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
finish_df = total_events_df[total_events_df['event_type'] == 'tutorial_finish']

#    Выборка первых регестраций для начала обучения
start_df_no_dupl = start_df.sort_values('start_time').drop_duplicates('user_id')

#    Выборка событий окончания первого обучения для тех обучений, которые были закончены
#    список с идентификаторами первых обучений > list_ids
#    фильтрация датафрейма finish_df_no_dupl > isin()
finish_df_no_dupl = total_events_df[total_events_df["event_type"] == 'tutorial_finish']
list_ids = start_df_no_dupl["tutorial_id"].unique()
finish_df_no_dupl = finish_df_no_dupl[finish_df_no_dupl["tutorial_id"].isin(list_ids)]


# 2. Проверка дублировния данных
'''
print('reg :', reg_df['user_id'].value_counts().mean())
print('level ch. :', level_ch_df['user_id'].value_counts().mean())
print('pack ch. :', pack_ch_df['user_id'].value_counts().mean())
print('purch :', purch_df['user_id'].value_counts().mean())
print('start :', start_df['user_id'].value_counts().mean().round(2))
print('start без дубликатов :', start_df_no_dupl['user_id'].value_counts().mean())
print('finish :', finish_df['user_id'].value_counts().mean().round(2))
print('finish без дубликатов:', tutorial_finish_df['user_id'].value_counts().mean().round(2))
'''

# 3. Переименование признака'start_time'
reg_df = reg_df[['user_id', 'start_time']].rename(columns={'start_time':'reg_time'})
level_ch_df = level_ch_df[['user_id', 'start_time']].rename(columns={'start_time':'level_time'})
pack_ch_df = pack_ch_df[['user_id', 'start_time']].rename(columns={'start_time':'puck_time'})
purch_df = purch_df[['user_id', 'start_time']].rename(columns={'start_time': 'purch_time'})
finish_df_no_dupl = finish_df_no_dupl[['user_id', 'start_time']].rename(columns={'start_time': 'finish_time'})
start_df_no_dupl = start_df_no_dupl[['user_id', 'tutorial_id', 'start_time']].rename(columns={'start_time':'tut_start_time'})

# 4. Объединение. Вычисление временных промежутков

#    Датафреймы: Регестрация / Начало обучения
merged_df1 = reg_df.merge(start_df_no_dupl, on='user_id', how='inner')
merged_df1['timedelta1'] = (merged_df1['tut_start_time'] - merged_df1['reg_time'])
print('▪ timedelta1 - время между Регестрацией и Началом обучения пользователей:')
merged_df1.sort_values('timedelta1').head(3)
display(merged_df1['timedelta1'].describe())

#   Датафреймы: Начало обучения / Конец обучения :
merged_df2 = start_df_no_dupl.merge(finish_df_no_dupl, on='user_id', how='inner')
merged_df2['timedelta2'] = (merged_df2['finish_time'] - merged_df2['tut_start_time'])
print('▪ timedelta2 - время между Началом и Завершением обучения пользователей:')
merged_df2.sort_values('timedelta2').head(3)
display(merged_df2['timedelta2'].describe())


#   Датафреймы: Регистрация / Выбор уровня сложности :
merged_df3 = reg_df.merge(level_ch_df, on='user_id', how='inner')
merged_df3['timedelta3'] = (merged_df3['level_time'] - merged_df3['reg_time'])
print('▪ timedelta3 - время между Регестрацией и Выбором уровня сложности:')
merged_df3.sort_values('timedelta3').head(3)
display(merged_df3['timedelta3'].describe())

#   Датафреймы: Выбор уровня сложности / Выбор пакета бесплатных вопросов:
merged_df4 = level_ch_df.merge(pack_ch_df, on='user_id', how='inner')
merged_df4['timedelta4'] = (merged_df4['puck_time'] - merged_df4['level_time'])
print('▪ timedelta4 - время между Выбором уровня сложности и Выбором пакета бесплатных вопросов:')
merged_df4.sort_values('timedelta4').head(3)
display(merged_df4['timedelta4'].describe())
timedelta4_mean = merged_df4['timedelta4'].mean()

#   Датафреймы: Выбор пакета бесплатных вопросов / Оплата:
merged_df5 = pack_ch_df.merge(purch_df, on='user_id', how='inner')
merged_df5['timedelta5'] = (merged_df5['purch_time'] - merged_df5['puck_time'])
print('▪ timedelta5 - время между Выбором пакета бесплатных вопросов и Оплатой:')
merged_df5.sort_values('timedelta5').head(3)
display(merged_df5['timedelta5'].describe())

print('\n РЕЗЮМИРУЕМ: ')
print(f'▪ 75 % пользователей проходят обучение за 5 мин 7 сек.')
print(f'▪ 5 мин 17 сек - среднее время между выбором уровня сложности и выбором набора бесплатных вопросов для всех пользователей')
print(f'▪ 3 дня 17 ч 46 мин -  среднее время между выбором бесплатных вопросов и первой оплатой')


▪ timedelta1 - время между Регестрацией и Началом обучения пользователей:


count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta1, dtype: object

▪ timedelta2 - время между Началом и Завершением обучения пользователей:


count                         9830
mean     0 days 00:03:53.143234994
std      0 days 00:01:44.754559741
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%         0 days 00:05:07.750000
max                0 days 00:10:06
Name: timedelta2, dtype: object

▪ timedelta3 - время между Регестрацией и Выбором уровня сложности:


...

▪ timedelta4 - время между Выбором уровня сложности и Выбором пакета бесплатных вопросов:


count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta4, dtype: object

▪ timedelta5 - время между Выбором пакета бесплатных вопросов и Оплатой:


count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta5, dtype: object


 РЕЗЮМИРУЕМ: 
▪ 75 % пользователей проходят обучение за 5 мин 7 сек.
▪ 5 мин 17 сек - среднее время между выбором уровня сложности и выбором набора бесплатных вопросов для всех пользователей
▪ 3 дня 17 ч 46 мин -  среднее время между выбором бесплатных вопросов и первой оплатой


### ПРОВЕРКА ГИПОТЕЗ
определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей(П) :
- П, которые прошли обучение хотя бы раз
- П, которые начали обучение, но не прошли его ни разу
- П, которые не начинали обучение, а сразу же перешли к выбору уровня сложности

In [11]:
# ПРОВЕРКА ГИПОТЕЗЫ

# 1.  set1 : П, прошедше обучение хотя бы раз (10250 чел.)
#     список пользователей, у которых есть событие tutorial_finish:
users_finished = total_events_df[total_events_df['event_type'] == 'tutorial_finish']['user_id'].unique()
set_finised = set(users_finished)
len_finished = len(users_finished)


# 2. set2: П, которые начали, но не закончили обучение (1608 чел.)
#    список пользователей, у которых есть  tutorial_start но нет tutorial_finish:
users_started = total_events_df[total_events_df['event_type'] == 'tutorial_start']['user_id'].unique()
set_started = set(users_started)
len_started = len(set_started)
set_started_not_finished = set_started.difference(set_finised)
len_started_not_finished = len(set_started_not_finished)
# OR: 11858 - 10250 = 1608
set2 = len_started - len_finished


# 3. set3 : П, которые не начинали обучение, а сразу выбрали уровень сложности (8068 чел.)
#    список пользователей, у которых отсутствует tutorial_start:
all_users = total_events_df['user_id'].unique()
set_all_users = set(all_users)
len_all_users = len(set_all_users)
users_not_started = set_all_users.difference(set_started)
len_not_started = len(users_not_started)
# OR:  19926 - 11858 = 8068
set3 = len_all_users - len_started


# 4. Пользователи, которые завершили обучение и совершили оплату (14.12%)
purchase_df_1 = purchase_df[purchase_df['user_id'].isin(users_finished)]
count_purch1 = purchase_df_1['user_id'].nunique() # 1447
percent_of_purchase_1 = count_purch1 / len_finished
#    Cредний чек (110.99)
mean_purch1 = purchase_df_1['amount'].mean()



# 5. Пользователи, которые начали и не закончивших обучение, но совешшили оплату (8.15%)
purchase_df_2 = purchase_df[purchase_df['user_id'].isin(set_started_not_finished)]
count_purch2 = purchase_df_2['user_id'].nunique() # 131
percent_of_purchase_2 = count_purch2 / len_started_not_finished
#    Средний чек (104.96)
mean_purch2 = purchase_df_2['amount'].mean()



# 6. Пользователи, ни разу не начинавших обучение, но совершившие оплату (0.27%)
purchase_df_3 = purchase_df[purchase_df['user_id'].isin(users_not_started)]
count_purch3 = purchase_df_3['user_id'].nunique() # 22
percent_of_purchase_3 = count_purch3 / len_not_started
#    Средний чек (128.42)
mean_purch3= purchase_df_3['amount'].mean()


print('\n РЕЗЮМИРУЕМ: ')
print(f'▪ {len_finished} пользователей прошли обучение хотя бы раз')
print(f'▪ {len_started_not_finished} пользователей начали обучение, но не прошли его ни разу')
print(f'▪ {len_not_started} пользователей не начинали обучение, а сразу же перешли к выбору уровня сложности')
print(f'▪ {count_purch1} пользователей оплатили тренировки это {percent_of_purchase_1:.2%} от числа пользователей, завершивших обучение')
print(f'▪ {mean_purch1.round(2)} - средний размер платежа пользователей, завершивших обучение')
print(f'▪ {count_purch2} пользователь оплатил тренировки это {percent_of_purchase_2:.2%} от числа пользователей, начавших обучение, но не завершивших')
print(f'▪ {mean_purch2.round(2)} - средний размер платежа пользователей, не завершивших обучение')
print(f'▪ {count_purch3} пользователь оплатил тренировки это {percent_of_purchase_3:.2%} от числа пользователей, ни разу не начинавших обучение')
print(f'▪ {mean_purch3.round(2)} - средний размер платежа пользователей, не начавших обучение\n')


 РЕЗЮМИРУЕМ: 
▪ 10250 пользователей прошли обучение хотя бы раз
▪ 1608 пользователей начали обучение, но не прошли его ни разу
▪ 8068 пользователей не начинали обучение, а сразу же перешли к выбору уровня сложности
▪ 1447 пользователей оплатили тренировки это 14.12% от числа пользователей, завершивших обучение
▪ 110.99 - средний размер платежа пользователей, завершивших обучение
▪ 131 пользователь оплатил тренировки это 8.15% от числа пользователей, начавших обучение, но не завершивших
▪ 104.96 - средний размер платежа пользователей, не завершивших обучение
▪ 22 пользователь оплатил тренировки это 0.27% от числа пользователей, ни разу не начинавших обучение
▪ 128.41 - средний размер платежа пользователей, не начавших обучение



Пользователей, завершивших обучение и совершили оплату, больше, чем пользователей, которые не начали обучение или не закончили его.
Факт прохождения обучения влияет на дальнейшую мотивацию использовать приложение.
Существенной разницы в среднем чеке не наблюдается, то есть обучение увеличивает вовлечённость, но выбор количества платных пакетов зависит индивидуально от пользователя.
